In [23]:
from transformers import AutoModelForCausalLM
import torch

olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-7B-0724-hf", revision="step0-tokens0B")

Loading checkpoint shards: 100%|██████████| 6/6 [00:01<00:00,  4.31it/s]


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-7B-0724-hf", )

In [9]:
inputs = tokenizer("Hi Oh Hi Mark!", return_tensors='pt', return_token_type_ids=False)


In [20]:
from huggingface_hub import list_repo_refs
out = list_repo_refs("allenai/OLMo-7B-0424-hf")
branches = [b.name for b in out.branches]
branches

['main',
 'nitro',
 'step651581-tokens2731B',
 'step650650-tokens2728B',
 'step649650-tokens2723B',
 'step648650-tokens2719B',
 'step647650-tokens2715B',
 'step646650-tokens2711B',
 'step645650-tokens2707B',
 'step644650-tokens2702B',
 'step643650-tokens2698B',
 'step642650-tokens2694B',
 'step641650-tokens2690B',
 'step640650-tokens2686B',
 'step639650-tokens2681B',
 'step639000-tokens2679B',
 'step638000-tokens2675B',
 'step637000-tokens2670B',
 'step636000-tokens2666B',
 'step635000-tokens2662B',
 'step634000-tokens2658B',
 'step633000-tokens2654B',
 'step632000-tokens2649B',
 'step631000-tokens2645B',
 'step630000-tokens2641B',
 'step629000-tokens2637B',
 'step628000-tokens2633B',
 'step627000-tokens2628B',
 'step626000-tokens2624B',
 'step625000-tokens2620B',
 'step624000-tokens2616B',
 'step623000-tokens2612B',
 'step622000-tokens2607B',
 'step621100-tokens2604B',
 'step621000-tokens2603B',
 'step620000-tokens2599B',
 'step619000-tokens2595B',
 'step618000-tokens2591B',
 'step617

In [11]:
response = olmo.generate(**inputs, max_new_tokens=2, do_sample=True, top_k=50, top_p=0.95)
print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

Hi Oh Hi Mark!
Wow


In [16]:


with torch.no_grad():
    logits = olmo.forward(inputs['input_ids'])

In [22]:
# compute cross-entropy loss
target = inputs['input_ids'][:, 1:]
loss = torch.nn.functional.cross_entropy(logits.logits[:, :-1].reshape(-1, logits.logits.shape[-1]), target.reshape(-1))

print(loss.item())

6.922701835632324


In [20]:
logits.logits.shape

torch.Size([1, 5, 50304])

In [21]:
inputs['input_ids'].shape

torch.Size([1, 5])

In [24]:
olmo.parameters()

<generator object Module.parameters at 0x7de275c09620>

In [24]:
def compute_model_norm(model):
    with torch.no_grad():
        # compute the L2 norm without keeping all model parameters in memory
        norm = []
        for p in model.parameters():
            norm.append(torch.linalg.vector_norm(p.view(-1), 2).item() ** 2)
    return torch.Tensor(norm).sum().sqrt().item()
    
compute_model_norm(olmo)

1082.3917236328125

In [ ]:
# model norm

# step 0: 1082.3917236328125

# final 7B: 359.68658447265625

